## Librerias

In [1]:
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
import shap
import random
from scipy import stats
from tqdm import tqdm
import statsmodels.api as sm
import os

import re

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, log_loss, make_scorer

from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import lightgbm as lgb
import xgboost as xgb

random.seed(123)

#IMPORTAR FUNCIONES DE INTERÉS
from trees_models import  xgboost_regression, random_forest_regression, catboost_regression, lightgbm_regression

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [16]:
import statsmodels.api as sm

In [15]:
model = sm.OLS(Y, X)
results = model.fit()

,Explicativa,CAT_FE,CAT_SHAP,CAT_SHAP_ABS,CAT_MAE_OUT,CAT_MAE_IN,CAT_coef,CAT_PARAM
9,defence_fdh,22.722745,-0.000700,0.000700,0.112055,0.078119,0.2178***,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
5,housing_fdh,16.981815,-0.000180,0.000180,0.112055,0.078119,0.1186**,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
10,general_fdh,11.715212,0.000872,0.000872,0.112055,0.078119,0.0664,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
7,economy_fdh,9.924588,-0.001986,0.001986,0.112055,0.078119,0.0927**,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
0,energy_fdh,8.529212,0.000439,0.000439,0.112055,0.078119,0.0176,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
3,education_fdh,7.679006,0.001354,0.001354,0.112055,0.078119,0.1181**,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
6,environment_fdh,6.518073,-0.000568,0.000568,0.112055,0.078119,0.1087***,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
4,health_fdh,5.206999,0.000102,0.000102,0.112055,0.078119,0.0171,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
1,transportation_fdh,4.775452,-0.000333,0.000333,0.112055,0.078119,0.0447,"{'iterations': 60, 'depth': 5, 'learning_rate'..."
8,order_fdh,4.259010,0.000919,0.000919,0.112055,0.078119,0.0775*,"{'iterations': 60, 'depth': 5, 'learning_rate'..."


In [2]:
def mejor_modelo(modelos, out):
    """
    Retorna el DataFrame con el valor "out" más bajo.

    Parámetros:
    - modelos: lista de DataFrames.
    - out: lista de valores "out" asociados a cada DataFrame.
    
    Retorna:
    - DataFrame asociado al valor "out" más bajo.
    """
    # Encontrar el índice del valor mínimo en 'out'
    indice_minimo = out.index(min(out))
    
    # Retornar el DataFrame correspondiente
    return modelos[indice_minimo]

### ORGANIZACIÓN DE LOS DATOS + MODELO PRINCIPAL

In [12]:
# version = "23_08_15_V1"
# version = "23_08_22_V2"
df = pd.read_excel(f"(20240122)time_levelex(inputorientation) - copia.xlsx")
columnas = list(df.columns)
fdhs = [i for i in columnas if i.endswith('_fdh')]
ids = ["Country Code", "Time"]

In [13]:
dfr = df[ids+fdhs]

nans = dfr.isna().sum()
drop = list(nans[nans>70].index)
for i in drop:
    fdhs.remove(i)

dfr = df[ids+fdhs]

dfp = dfr.dropna()

In [14]:
features = dfp[fdhs[:-1]]
target = dfp[fdhs[-1]]
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.1, random_state=2)


trials_n = 20
fperdida = "MAE"


g = xgboost_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)
rf = random_forest_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)
cb = catboost_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)

modelos = [cb, g, rf]
out = cb["CAT_MAE_OUT"].iloc[0], g["XGB_MAE_OUT"].iloc[0], rf["RF_MAE_OUT"].iloc[0]

BM = mejor_modelo(modelos, out)

[I 2024-03-19 19:19:15,112] A new study created in memory with name: no-name-ab586edf-fb17-4dfd-b7e9-9e1ecfb4cd7f
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
[I 2024-03-19 19:19:15,640] Trial 0 finished with value: 0.3306868836692143 and paramete

In [9]:
with pd.ExcelWriter('RESULTADOS_ML.xlsx') as writer:  
    g.to_excel(writer, sheet_name='G')
    cb.to_excel(writer, sheet_name='CB')
    rf.to_excel(writer, sheet_name='RF')

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).


In [10]:
BMT = BM.iloc[0:-1, [0,4,6,1]]

tabla_total = dfr.groupby("Country Code").sum()
tabla_total = tabla_total.sort_values("Total_fdh", ascending = False).iloc[:, 1:].T
tabla_total = tabla_total.reset_index().rename(columns = {"index": "Explicativa"})
tabla_total = pd.merge(BMT, tabla_total, "outer", "Explicativa")
# tabla_total.to_excel(f"TABLA_GENERAL_ML.xlsx", index = False)

In [12]:
tabla_total.to_excel("prueba.xlsx")

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).


### MODELO SECTORIAL

In [10]:
# Función para limpiar el nombre de la columna
def clean_column_name(name):
    # Quitar contenido entre paréntesis y corchetes
    name = re.sub(r'\[.*?\]|\(.*?\)', '', name)
    
    # Quitar caracteres que no sean alfanuméricos y espacios
    name = re.sub(r'[^a-zA-Z0-9 ]', '', name)
    
    # Quitar '_fdh' si está presente
    # name = name.replace('_fdh', '')
    
    # Convertir a título y reemplazar espacios con guiones bajos
    name = name.strip().title().replace(' ', '_')
    
    return name


## Modelos por catgoría

In [11]:
xls = pd.ExcelFile(f"{version}//general.xlsx")

sheet_names = xls.sheet_names

trials_n = 20
fperdida = "MAE"

NameError: name 'version' is not defined

In [13]:
SD = pd.DataFrame()


time_series = f"{version}//4_ML"
if not os.path.exists(time_series):
    os.mkdir(time_series)

for name in sheet_names[1:]:
    dfr = pd.read_excel(f"{version}//general.xlsx", sheet_name=name)
    fdhs = [i for i in list(dfr.columns) if i.endswith('_fdh')]

    dfr2 = dfr[fdhs]

    try:

        nans = dfr2.isna().sum()
        drop = list(nans[nans>60].index)
        for i in drop:
            fdhs.remove(i)

        dfr2 = dfr[fdhs]

        dfp = dfr.dropna()

        fdhs.remove(f"{name}_fdh")
        features = dfp[fdhs.copy()]
        target = dfp[f"{name}_fdh"]

        # Aplicar la función de limpieza a los nombres de las columnas
        features.columns = [clean_column_name(col) for col in features.columns]
        features = features.loc[:, ~features.columns.duplicated()]

        X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.1, random_state=2)

        g = xgboost_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)
        rf = random_forest_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)
        cb = catboost_regression(X_train, Y_train, X_test, Y_test, fperdida, trials_n)

        modelos = [cb, g, rf]
        out = cb["CAT_MAE_OUT"].iloc[0], g["XGB_MAE_OUT"].iloc[0], rf["RF_MAE_OUT"].iloc[0]

        BM = mejor_modelo(modelos, out)
        BM.to_excel(f"{version}//4_ML//ML_{name}.xlsx")

        BM = BM.iloc[:-1, [0,1]]
        BM.columns = ["target", "value"]
        BM["source"] = name

        # BM
    
    except Exception as e:
        print("Ocurrió un error:", str(e))
        BM = pd.DataFrame()


    SD = pd.concat([SD, BM])

[I 2023-08-24 08:40:11,846] A new study created in memory with name: no-name-be83779e-9eb6-4844-8506-506cafd83860
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
[I 2023-08-24 08:40:12,839] Trial 0 finished with value: 0.3445349396488259 and paramete

In [15]:
SD.to_excel(f"{version}//TABLE_GENERAL_ML_CAT.xlsx")

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).


## Graficas

In [10]:
tabla_total = pd.read_excel(f"{version}/TABLA_GENERAL_ML.xlsx")

In [11]:
T = tabla_total.iloc[:-1, [0,3]]
T.columns = ["target", "value"]
T["target"]  = T["target"].apply(lambda x: x.replace("_fdh", ""))
T["source"] = "total"

In [12]:
SD = pd.read_excel(f"{version}//TABLE_GENERAL_ML_CAT.xlsx")

In [13]:
SD = pd.concat([T,SD])

In [14]:
SD.source = [
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "Total",
    "General",
    "General",
    "General",
    "Defence",
    "Order",
    "Economy",
    "Economy",
    "Economy",
    "Economy",
    "Economy",
    "Economy",
    "Environment",
    "Environment",
    "Environment",
    "Housing",
    "Health",
    "Health",
    "Health",
    "Health",
    "Health",
    "Health",
    "Education",
    "Education",
    "Education",
    "Education",
    "Social",
    "Social",
    "Social",
    "Social",
    "Transportation",
    "Transportation",
    "Transportation",
    "Transportation",
    "Energy",
    "Energy",
    "Energy",
    "Energy",
    "Energy"
]

In [15]:
SD.target = [
    "Defence",
    "Education",
    "Health",
    "Social",
    "General",
    "Housing",
    "Environment",
    "Energy",
    "Order",
    "Transportation",
    "Economy",
    "Safe Drinking Water Usage",
    "Population Living in Slums",
    "Access to Electricity",
    "Political Stability",
    "Voice and Accountability",
    "GDP Growth",
    "GDP per Capita",
    "Inflation Variability",
    "Statistical Capacity",
    "Growth Variability",
    "Unemployment Rate",
    "Forest Area",
    "CO2 Emissions",
    "Air Pollution Exposure",
    "Improved Housing Rate",
    "CVD, Cancer, Diabetes Survival",
    "Maternal Mortality Risk",
    "Life Expectancy",
    "Infant Mortality Rate",
    "Maternal Mortality Ratio",
    "Stunting Prevalence",
    "Secondary School Enrollment",
    "Educational System Quality",
    "Education Performance",
    "Primary Completion Rate",
    "Multidimensional Poverty",
    "Informal Economy",
    "National Poverty Rate",
    "Gini Index",
    "Logistics Performance",
    "Road Traffic Mortality",
    "Trade Efficiency",
    "Infrastructure Quality",
    "Energy Usage",
    "Access to Electricity (Urban)",
    "Fossil Fuel Consumption",
    "Net Energy Imports",
    "Renewable Energy Consumption"
]


In [19]:
SD.to_excel(f"{version}//TABLA_GRAFICA_PLOTLY.xlsx", index = False)

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).


In [20]:
SD

,target,value,source,Unnamed: 0
0,Defence,16.975993,Total,NaN
1,Education,11.341545,Total,NaN
2,Health,11.282190,Total,NaN
3,Social,11.186658,Total,NaN
4,General,10.169585,Total,NaN
5,Housing,8.248822,Total,NaN
6,Environment,7.815834,Total,NaN
7,Energy,6.677748,Total,NaN
8,Order,6.317300,Total,NaN
9,Transportation,5.735271,Total,NaN


In [23]:
import plotly.graph_objects as go

In [24]:
import plotly.express as px

# Asumiendo que tu DataFrame 'SD' ya está definido anteriormente...

# Crear etiquetas para cada nodo único colocando primero las etiquetas 'target'
labels = list(SD['target'].unique()) + list(SD['source'].unique())

# Crear listas de fuentes y objetivos
sources = SD['source'].apply(lambda x: labels.index(x))
targets = SD['target'].apply(lambda x: labels.index(x))

# Crear lista de valores
values = SD['value']

# Definir gamas de colores para las categorías.
# Obtener categorías únicas desde la columna 'source'
unique_sources = SD['source'].unique()


# Definir los datos para el diagrama de Sankey
sankey_data = dict(
    label = labels,
    source = sources, 
    target = targets,
    value = values
)

# Crear el objeto de figura y añadir el diagrama de Sankey
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 30,
      thickness = 70,
      line = dict(color = "black", width = 1),
      label = sankey_data['label']
      # color = colors
    ),
    link = dict(
      source = sankey_data['source'],
      target = sankey_data['target'],
      value = sankey_data['value']
    )
)])

# Ajustar las dimensiones del layout
fig.update_layout(
    width=1500,
    height=1200   
)

# Mostrar el diagrama
fig.show()


In [21]:
import chart_studio
chart_studio.tools.set_credentials_file(username='jorgealbertoguerra18', api_key='uosUmZZXDGL6o1Ahv3NE')

import chart_studio.plotly as py
import plotly.express as px

# Extract the main categories from SD DataFrame
main_categories = SD[SD["source"] == "Total"]

# Extract the sub-categories from SD DataFrame
sub_categories = SD[SD["source"] != "Total"]

# Create a total sum for all categories
all_categories_value = SD["value"].sum()

# Create lists for 'character', 'parent', and 'value' for Plotly sunburst
character = ["All Categories"] + main_categories["target"].tolist() + sub_categories["target"].tolist()
parent = [""] + ["All Categories"]*len(main_categories) + sub_categories["source"].tolist()
value = [all_categories_value] + main_categories["value"].tolist() + sub_categories["value"].tolist()

# Create the DataFrame for Plotly
data = {
    'character': character,
    'parent': parent,
    'value': value
}

# Create the sunburst chart
fig = px.sunburst(data, names='character', parents='parent', values='value', width=1100, height=1200)

# Upload and display the chart on Plotly platform
py.plot(fig, filename='sunburst_chart', auto_open=True)


'https://plotly.com/~jorgealbertoguerra18/3/'